In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
    builder.\
    master("local").\
    appName('logistic-regression-project').\
    getOrCreate()

In [2]:
df = spark.read.csv(
    path='D:/learn-ab/learning-PySpark/sample-data/customer-churn.csv',
    inferSchema=True,
    header=True
)
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [3]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [4]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                NULL|                NULL|0.16666666666666666|
| stddev|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [5]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
assembler = VectorAssembler(
    inputCols=[
        'Age',
        'Total_Purchase',
        'Account_Manager',
        'Years',
        'Num_Sites'
    ],
    outputCol='features'
)
df_final = assembler.transform(df).\
    select('features', 'churn')
df_final.show()

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [8]:
train_data, test_data = df_final.randomSplit([0.7, 0.3])
train_data.describe().show()
test_data.describe().show()

+-------+-------------------+
|summary|              churn|
+-------+-------------------+
|  count|                619|
|   mean|0.16962843295638125|
| stddev|0.37560956830021414|
|    min|                  0|
|    max|                  1|
+-------+-------------------+

+-------+------------------+
|summary|             churn|
+-------+------------------+
|  count|               281|
|   mean|0.1601423487544484|
| stddev|0.3673923277936089|
|    min|                 0|
|    max|                 1|
+-------+------------------+



In [9]:
from pyspark.ml.classification import LogisticRegression

In [10]:
?LogisticRegression

Init signature:
LogisticRegression(
    *,
    featuresCol: str = 'features',
    labelCol: str = 'label',
    predictionCol: str = 'prediction',
    maxIter: int = 100,
    regParam: float = 0.0,
    elasticNetParam: float = 0.0,
    tol: float = 1e-06,
    fitIntercept: bool = True,
    threshold: float = 0.5,
    thresholds: Optional[List[float]] = None,
    probabilityCol: str = 'probability',
    rawPredictionCol: str = 'rawPrediction',
    standardization: bool = True,
    weightCol: Optional[str] = None,
    aggregationDepth: int = 2,
    family: str = 'auto',
    lowerBoundsOnCoefficients: Optional[pyspark.ml.linalg.Matrix] = None,
    upperBoundsOnCoefficients: Optional[pyspark.ml.linalg.Matrix] = None,
    lowerBoundsOnIntercepts: Optional[pyspark.ml.linalg.Vector] = None,
    upperBoundsOnIntercepts: Optional[pyspark.ml.linalg.Vector] = None,
    maxBlockSizeInMB: float = 0.0,
)
Docstring:     
Logistic regression.
This class supports multinomial logistic (softmax) and binom

In [11]:
log_reg = LogisticRegression(
    featuresCol='features',
    labelCol='churn'
)
log_reg_model = log_reg.fit(train_data)
log_reg_model_summary = log_reg_model.summary

In [12]:
log_reg_model_summary.predictions.describe().show()

+-------+-------------------+------------------+
|summary|              churn|        prediction|
+-------+-------------------+------------------+
|  count|                619|               619|
|   mean|0.16962843295638125|0.1260096930533118|
| stddev|0.37560956830021414|0.3321286741860335|
|    min|                0.0|               0.0|
|    max|                1.0|               1.0|
+-------+-------------------+------------------+



In [13]:
pred_and_labels = log_reg_model.evaluate(test_data)
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8939.61,0.0...|    0|[6.18032267755672...|[0.99793451517581...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.30699904617236...|[0.98670520938679...|       0.0|
|[29.0,13240.01,1....|    0|[6.45844656169183...|[0.99843522394845...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.06430927607732...|[0.95539629416520...|       0.0|
|[30.0,10960.52,1....|    0|[2.22029705761465...|[0.90205744382154...|       0.0|
|[30.0,13473.35,0....|    0|[2.59814226631730...|[0.93074192333488...|       0.0|
|[31.0,5304.6,0.0,...|    0|[3.30280901985182...|[0.96452505100729...|       0.0|
|[31.0,7073.61,0.0...|    0|[2.99645559962407...|[0.95241374555238...|       0.0|
|[31.0,8829.83,1.0...|    0|[4.20075638701755...|[0.98523697405454...|       0.0|
|[31.0,10058.87,

In [14]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [15]:
?BinaryClassificationEvaluator

Init signature:
BinaryClassificationEvaluator(
    *,
    rawPredictionCol: str = 'rawPrediction',
    labelCol: str = 'label',
    metricName: 'BinaryClassificationEvaluatorMetricType' = 'areaUnderROC',
    weightCol: Optional[str] = None,
    numBins: int = 1000,
)
Docstring:     
Evaluator for binary classification, which expects input columns rawPrediction, label
and an optional weight column.
The rawPrediction column can be of type double (binary 0/1 prediction, or probability of label
1) or of type vector (length-2 vector of raw predictions, scores, or label probabilities).

.. versionadded:: 1.4.0

Examples
--------
>>> from pyspark.ml.linalg import Vectors
>>> scoreAndLabels = map(lambda x: (Vectors.dense([1.0 - x[0], x[0]]), x[1]),
...    [(0.1, 0.0), (0.1, 1.0), (0.4, 0.0), (0.6, 0.0), (0.6, 1.0), (0.6, 1.0), (0.8, 1.0)])
>>> dataset = spark.createDataFrame(scoreAndLabels, ["raw", "label"])
...
>>> evaluator = BinaryClassificationEvaluator()
>>> evaluator.setRawPredictionCol(

In [16]:
log_reg_model_eval = BinaryClassificationEvaluator(
    labelCol='churn',
    rawPredictionCol='prediction'
)
auc = log_reg_model_eval.evaluate(pred_and_labels.predictions)

In [17]:
auc

0.8052730696798494

In [18]:
new_cust = spark.read.csv(
    path='D:/learn-ab/learning-PySpark/sample-data/new-customers.csv',
    inferSchema=True,
    header=True
)
new_cust.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [19]:
new_cust.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [20]:
new_cust_final = assembler.transform(new_cust)
new_cust_final.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   

In [21]:
new_cust_final.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [22]:
new_cust_final.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         NULL|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                NULL|            NULL|
| stddev|         NULL| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                NULL|            NULL|
|    min|Andrew Mccall|              22.0|            100.0|          

In [23]:
final_log_reg_model = log_reg.fit(df_final)
new_cust_res = final_log_reg_model.transform(new_cust_final)
new_cust_res.select('Names', 'Company', 'prediction').show()

+--------------+----------------+----------+
|         Names|         Company|prediction|
+--------------+----------------+----------+
| Andrew Mccall|        King Ltd|       0.0|
|Michele Wright|   Cannon-Benson|       1.0|
|  Jeremy Chang|Barron-Robertson|       1.0|
|Megan Ferguson|   Sexton-Golden|       1.0|
|  Taylor Young|        Wood LLC|       0.0|
| Jessica Drake|   Parks-Robbins|       1.0|
+--------------+----------------+----------+

